In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База Сентябрь\База Сентябрь 08-09.xlsx")

In [3]:
df['Дата обновления'] = pd.to_datetime(df['Дата обновления'], errors='coerce')

In [4]:
# Шаг 1 — исключаем квартиры от Фонда Реновации
df_filtered = df[df['Девелопер'] != 'Фонд реновации'].copy()

# Шаг 2 — добавляем числовой месяц
df_filtered['месяц'] = df_filtered['Дата обновления'].dt.month

# Шаг 3 — группировка по проекту и девелоперу + подсчёт квартир по месяцам
project_month_counts = (
    df_filtered
    .groupby(['Название проекта', 'Девелопер', 'месяц'])
    .size()
    .unstack(fill_value=0)
    .rename(columns={8: 'август', 9: 'сентябрь'})
)
project_month_counts = project_month_counts.rename(columns={8: 'август', 9: 'сентябрь'})

# Шаг 4 — фильтрация:
# - минимум 5 квартир в июне
# - июль не ноль
# - изменение >= 80% в любую сторону
filtered = project_month_counts[
    (project_month_counts['август'] >= 1) &
    (project_month_counts['сентябрь'] >= 0) &
    (
        (project_month_counts['сентябрь'] >= project_month_counts['август'] * 2) |
        (project_month_counts['сентябрь'] <= project_month_counts['август'] * 0.2)
    )
]

# Шаг 5 — читаемый вывод
result = [
    f"{project} — август: {row['август']}, сентябрь: {row['сентябрь']}"
    for project, row in filtered.iterrows()
]

# Вывод
for line in result:
    print(line)

('Armani/Casa Moscow Residences', 'VOS’HOD') — август: 6, сентябрь: 0
('DIUS', ' ') — август: 1, сентябрь: 0
('Nagatino i-Land', 'Эталон') — август: 79, сентябрь: 0
('Vesper Кутузовский', 'Vesper') — август: 38, сентябрь: 93
('WOW', 'Страна Девелопмент') — август: 2, сентябрь: 5
('Апрелевка Клаб', 'ФСК') — август: 54, сентябрь: 0
('Берег', 'СЭСМА') — август: 188, сентябрь: 0
('Берег столицы', 'Ташир') — август: 3, сентябрь: 0
('Высокий берег', 'МОСОБЛСТРОЙ-17') — август: 220, сентябрь: 0
('Гармония парк', 'ТКС РИЭЛТИ') — август: 256, сентябрь: 0
('Гранд Комфорт', 'Зеленый сад') — август: 410, сентябрь: 0
('Десятка', 'Эталон') — август: 17, сентябрь: 3
('Квартал на воде', 'Самолет') — август: 19, сентябрь: 106
('Комплекс апартаментов LES', 'ПАТЕК Групп') — август: 4, сентябрь: 0
('Космос', 'СЗ Скала') — август: 932, сентябрь: 0
('Лесная Коллекция', 'Град') — август: 241, сентябрь: 0
('М_5', 'Садовое кольцо') — август: 9, сентябрь: 0
('Митино', 'Аквилон') — август: 2, сентябрь: 4
('Мишин